In [0]:
# 1. Widgets
dbutils.widgets.text("run_date", "2026-01-15")
v_date = dbutils.widgets.get("run_date")

from pyspark.sql.functions import col, sum, count, round

# 2. Read from Silver
silver_df = spark.table("ecommerce_silver")

# 3. Aggregate Metrics
gold_df = silver_df.filter(col("event_type") == "purchase") \
    .groupBy("brand", "main_category") \
    .agg(
        count("*").alias("total_transactions"),
        round(sum("price"), 2).alias("total_revenue")
    )

# 4. Save to Gold Table
gold_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce_gold_brand_metrics")

print(f"Gold layer complete. Processed {gold_df.count()} brand categories.")